### 연습문제 
1. csv 폴더에 있는 Sales Records.csv 파일 로드 
2. 'Order ID', 'Total Revenue', 'Total Cost', 'Total Profit', 'Order Priority' 컬럼을 제거 
3. 컬럼의 이름을 변경 -> ['대륙', '국가', '상품종류', '판매채널', '주문일자', '발송일자', '판매개수', '단가', '원가']
4. 대륙, 국가를 기준으로 오름차순 정렬 
5. 기존의 인덱스는 제거하고 인덱스를 재지정
6. 파생변수 생성 
    - 총단가 -> 판매개수 * 단가
    - 총원가 -> 판매개수 * 원가 
    - 총이윤 -> 총단가 - 총원가

In [ ]:
import pandas as pd

In [ ]:
## 파일 로드 
sales = pd.read_csv("../../csv/sales records.csv")

In [ ]:
sales.info()

In [ ]:
## 특정 컬럼을 제거 
## drop(  )함수를 이용
sales.drop(
    ['Order ID', 'Total Revenue', 
     'Total Cost', 'Total Profit', 'Order Priority'], 
    axis=1, 
    inplace = True)

In [ ]:
sales.head(1)

In [ ]:
## 컬럼의 이름을 변경 
## columns 변수에 데이터를 대입 
## rename() 함수를 이용하여 컬럼의 이름을 변경 

sales.columns = ['대륙', '국가', '상품종류', '판매채널', '주문일자', '발송일자', 
                 '판매개수', '단가', '원가']

In [ ]:
sales.head(1)

In [ ]:
## 대륙, 국가를 기준으로 오름차순 정렬 
sales.sort_values( ['대륙', '국가'] , inplace=True)

In [ ]:
## 인덱스를 초기화 기존의 인덱스는 제거 
sales.reset_index(drop = True, inplace=True)

In [ ]:
## 파생변수를 생성 
sales['총단가'] = sales['판매개수'] * sales['단가']
sales['총원가'] = sales['판매개수'] * sales['원가']
sales['총이윤'] = sales['총단가'] - sales['총원가']

In [ ]:
sales.head(3)

In [ ]:
## 데이터프레임의 그룹화 
## groupby({그룹화할 컬럼의 이름}).연산식함수
# sales.loc[:, ['대륙', '총이윤']]
# sales.iloc[:, [0, -1]]
group_data =  sales[ [ '대륙', '총이윤' ] ].groupby('대륙').sum()

In [ ]:
group_data.sort_values('총이윤', ascending=False)

In [ ]:
group_data2 = sales[['국가', '총이윤']].groupby('국가').sum()

In [ ]:
group_data2.sort_values('총이윤', ascending=False, inplace= True)

In [ ]:
group_data2.index

In [ ]:
## case1 for문을 이용하여 랭크를 찾아본다 
korea_rank = 1

for i in group_data2.index:
    # i가 의미하는것은? 국가명
    if i == "South Korea":
        break
    korea_rank += 1
print(korea_rank)

In [ ]:
# case2 reset_index() 함수를 이용하여 랭크
case2 = group_data2.reset_index()

In [ ]:
flag = case2['국가'] == 'South Korea'
case2.loc[ flag, ].index[0] + 1

In [ ]:
# case3 파생변수를 생성
group_data2['rank'] = range(1, len(group_data2)+1)

In [ ]:
group_data2.head()

In [ ]:
group_data2.loc['South Korea', 'rank']

In [ ]:
# case4 list 안에 내장함수를 이용
list(group_data2.index).index("South Korea") + 1

In [ ]:
## 대륙, 국가로 그룹화 
group_data3 =\
     sales[['대륙', '국가', '총이윤']].groupby( ['대륙', '국가'] ).sum()

In [ ]:
group_data3.stack()

In [ ]:
## pivot_table()
## pandas안에 내장되어있는 함수

group_data4 =  pd.pivot_table(
    sales, 
    index = ['대륙', '국가'], 
    columns= '상품종류',
    values = '총이윤', 
    aggfunc= ["sum", 'mean'], 
    margins= True
)

In [ ]:
## group_data4에서 Asia 데이터만 출력
group_data4.loc['Asia',]

In [ ]:
group_data4['sum']['Baby Food']

In [ ]:
group_data4.columns

In [76]:
group_data4.loc['Asia', ('sum', 'Baby Food')]

국가
Bangladesh       616571.52
Bhutan          2604036.90
Brunei          2404072.94
Cambodia        1511424.62
China           4285517.16
India           1903587.88
Indonesia        729494.60
Japan           3122639.50
Kazakhstan      3366507.34
Kyrgyzstan      2389118.78
Laos            1025126.84
Malaysia         322856.48
Maldives        3987200.84
Mongolia        2003186.42
Myanmar          682427.34
Nepal           1396776.06
North Korea     2972426.88
Philippines     3223484.22
Singapore       1463878.06
South Korea     3162421.40
Sri Lanka       2938109.00
Taiwan          3372929.96
Tajikistan      2138540.74
Thailand        1577663.88
Turkmenistan    4374475.24
Uzbekistan      3777938.46
Vietnam         2439637.00
Name: (sum, Baby Food), dtype: float64

In [80]:
## groupby 에서 연산식이 2개 이상인 경우 
group_data5 = sales[['대륙', '국가', '상품종류', '총이윤']]\
    .groupby(['대륙', '국가', '상품종류'])\
    .agg(['sum', 'mean'])

In [81]:
group_data5.unstack()

총이윤                                     \
                                      sum                                      
상품종류                            Baby Food  Beverages      Cereal     Clothes   
대륙                 국가                                                          
Asia               Bangladesh   616571.52   61997.94  6008793.93  1159837.92   
                   Bhutan      2604036.90  108179.28  2794925.91  3436992.00   
                   Brunei      2404072.94  260331.84  2471040.87  1625006.88   
                   Cambodia    1511424.62  385142.04  2237517.63  2781540.00   
                   China       4285517.16  486759.78  1424438.61  1131049.44   
...                                   ...        ...         ...         ...   
Sub-Saharan Africa The Gambia  3548545.48  198600.12  1530126.48  1371051.36   
                   Togo        2473188.00  544921.02  3047496.00  1638740.16   
                   Uganda      2967250.44  696259.26  3426572.61  1613256.48   
                   Zambia      4973791.96  153859.50  1939057.92  1195750.08   
                   Zimbabwe    2146017.82  488607.66  3068580.42  2370496.32   

                                                                             \
                                                                              
상품종류                            Cosmetics     Fruits   Household       Meat   
대륙                 국가                                                         
Asia               Bangladesh  4524966.75   56837.44  5182708.56  1794535.6   
                   Bhutan      4171315.17   66246.08  5093711.55   887343.6   
                   Brunei      2356807.85    9331.52  4576302.49  1316458.0   
                   Cambodia    4737957.50   17342.36  2669081.65  2238407.6   
                   China       8310812.13   32891.68  2180675.34   464635.6   
...                                   ...        ...         ...        ...   
Sub-Saharan Africa The Gambia  2511204.41   64686.81  4174241.51   776146.8   
                   Togo        3585547.14    8360.29  3278470.86  1001858.0   
                   Uganda      7794244.36   64412.07  2980488.32   985842.0   
                   Zambia      3845135.05   84123.46  4032873.82  1366279.2   
                   Zimbabwe    4989895.13  156295.73  1695086.44   551408.0   

                                                             ...  \
                                                             ...   
상품종류                          Office Supplies Personal Care  ...   
대륙                 국가                                        ...   
Asia               Bangladesh      4340222.50     732654.16  ...   
                   Bhutan          1150642.50     461580.14  ...   
                   Brunei          3895696.25      19872.58  ...   
                   Cambodia        3767300.00    1055276.60  ...   
                   China           3854412.50     898952.32  ...   
...                                       ...           ...  ...   
Sub-Saharan Africa The Gambia      2522853.75     353847.20  ...   
                   Togo            3025960.00      62800.36  ...   
                   Uganda          3830930.00     295407.28  ...   
                   Zambia          3908573.75     588133.14  ...   
                   Zimbabwe        2202683.75     463058.68  ...   

                                                                       \
                                        mean                            
상품종류                                  Cereal    Clothes     Cosmetics   
대륙                 국가                                                   
Asia               Bangladesh  667643.770000  386612.64  6.464238e+05   
                   Bhutan      698731.477500  381888.00  1.042829e+06   
                   Brunei      494208.174000  406251.72  5.892020e+05   
                   Cambodia    559379.407500  278154.00  6.768511e+05   
                   China       284887.722000  